In [6]:
import torch
from torch.utils.data import DataLoader
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper
from dataset.dataset_loader import dataset_loader
import keras_tuner

# Search for best hyperparameters for model 4 and 8: Batch size, and SGD params

# Initial steps
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

# No batch size is fixed
local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, shuffle=True, batch_size=32)

In [7]:
from utils.my_tuner import HistoryDeletingRandomSearch

search_project_name = "random-search-rich-structure"
search_directory = "cnn_search"
# Load previous hypertuner
previous_tuner = HistoryDeletingRandomSearch(
    None, overwrite=False,
    project_name=search_project_name,
    directory=search_directory
)

Reloading Tuner from cnn_search/random-search-rich-structure/tuner0.json


In [8]:
previous_tuner.results_summary(5)

Results summary
Results in cnn_search/random-search-rich-structure
Showing 5 best trials
Objective(name="default_objective", direction="min")

Trial 42 summary
Hyperparameters:
lr: 0.0001
momentum: 0.9
filters_0: 64
filters_1: 64
filters_2: 64
filters_3: 64
kernel_0: 3
kernel_1: 5
kernel_2: 5
kernel_3: 3
units_0: 128
units_1: 64
units_2: 64
dropout_0: False
dropout_1: False
dropout_2: False
convolution_layers: 2
hidden_layers: 2
Score: 0.3861699104309082

Trial 08 summary
Hyperparameters:
lr: 0.0001
momentum: 0.9
filters_0: 64
filters_1: 64
filters_2: 128
filters_3: 32
kernel_0: 5
kernel_1: 3
kernel_2: 3
kernel_3: 3
units_0: 256
units_1: 32
units_2: 64
dropout_0: False
dropout_1: False
dropout_2: False
convolution_layers: 1
hidden_layers: 1
Score: 0.3864298164844513

Trial 23 summary
Hyperparameters:
lr: 0.0001
momentum: 0.9
filters_0: 16
filters_1: 256
filters_2: 128
filters_3: 256
kernel_0: 5
kernel_1: 5
kernel_2: 3
kernel_3: 3
units_0: 64
units_1: 64
units_2: 128
dropout_0: False
dr

In [9]:
project_name = "random-search-best-40-hp-env"
directory = "cnn_search"

In [10]:
from models.simple_cnn.conv_net_family import TunableConvNetFamily

# Model 8
model_family = TunableConvNetFamily()
model_family.load_parameters(previous_tuner.get_best_hyperparameters(2)[0])

model_family.parameters_fixed = True

In [11]:
from keras_tuner import HyperParameters

learning_parameters = HyperParameters()
learning_parameters.Choice(name="batch_size", values=[8, 16, 32, 64], default=16)
learning_parameters.Float(name="lr", min_value=1e-5, max_value=1e-2, sampling='log', step=2),
learning_parameters.Float(name="momentum", min_value=0.5, max_value=1, sampling='reverse_log', step=2)

0.5

In [12]:
from models.zero_one_validation_loss import ZeroOneLoss, iter_0_1_loss
from models.structure.learning_parameters.sgd_learning_parameters import SgdLearningParametersTunable
from models.structure.tunable_hypermodel import TunableHyperModel
from utils.my_tuner import HistoryDeletingBayesianOptimization
import keras

batch_tuner = HistoryDeletingBayesianOptimization(
    TunableHyperModel(model_family, SgdLearningParametersTunable(learning_rate=1e-4),
                      (3, 224, 224), tune_batch=True, verbose=False),
    hyperparameters=learning_parameters,
    objective='val_loss',
    tune_new_entries=False,
    executions_per_trial=1,
    overwrite=False,
    directory=directory,
    metrics=['accuracy', iter_0_1_loss, ZeroOneLoss()],
    max_trials=15,
    project_name=project_name
)

In [13]:
import keras
import callbacks.threshold_stop_cb

batch_tuner.search(train_dataloader, epochs=12, validation_data=validation_dataloader, callbacks=[
    keras.callbacks.CSVLogger(f"{directory}/{project_name}/search.log", separator=",", append=True),
    callbacks.threshold_stop_cb.ThresholdStopCallback(0.6, 4),
])

Trial 15 Complete [00h 09m 13s]
val_loss: 0.3308117687702179

Best val_loss So Far: 0.3124878704547882
Total elapsed time: 02h 03m 58s
